In [72]:
import sys
sys.path.append('..')
import pipeline.main  
import importlib
importlib.reload(pipeline.main)
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from utils.google_drive_utils import upload_df_to_drive
from pipeline.data_transform import transform_dataframe
from pipeline.main import load_data_from_drive , save_data_to_postgres
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 

In [73]:
creds_path = 'credentials.json'
load_dotenv()
db_url = os.getenv("DB_URL")
engine = create_engine(db_url)

df = transform_dataframe(pd.read_sql("SELECT * FROM employee_attrition_dataset", engine))
df.head()

,employee_id,age,gender,marital_status,department,job_role,job_level,monthly_income,hourly_rate,years_at_company,years_in_current_role,years_since_last_promotion,work_life_balance,job_satisfaction,performance_rating,training_hours_last_year,overtime,project_count,average_hours_worked_per_week,absenteeism,work_environment_satisfaction,relationship_with_manager,job_involvement,distance_from_home,number_of_companies_worked,attrition
0,1,58,Female,Married,IT,Manager,1,15488,28,15,4,2,1,3,4,14,False,6,54,17,4,4,4,20,3,False
1,2,48,Female,Married,Sales,Assistant,5,13079,28,6,9,1,2,1,1,6,True,2,45,1,4,1,2,25,2,False
2,3,34,Male,Married,Marketing,Assistant,1,13744,24,24,14,8,3,2,1,77,True,6,34,2,3,4,4,45,3,False
3,4,27,Female,Divorced,Marketing,Manager,1,6809,26,10,8,2,3,5,4,49,False,9,48,18,2,3,1,35,3,False
4,5,40,Male,Divorced,Marketing,Executive,1,10206,52,29,10,1,2,5,1,64,False,3,33,0,4,1,3,44,3,False


In [74]:
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

In [75]:
def calcular_taxa_rotatividade(df: pd.DataFrame, coluna_grupo: str) -> pd.DataFrame:
    
    agrupado = df.groupby(coluna_grupo)
    total = agrupado['attrition'].count().rename('total_funcionarios')
    saidas = agrupado['attrition'].sum().rename('total_saidas')
    
    resumo = pd.concat([total, saidas], axis=1)
    resumo['taxa_rotatividade_%'] = (resumo['total_saidas'] / resumo['total_funcionarios']) * 100
    
    resumo = resumo.reset_index()
    return resumo.sort_values(by='taxa_rotatividade_%', ascending=False)

In [81]:
df_departamento = calcular_taxa_rotatividade(df, 'department')
print("\n--- Tabela 1.1: Rotatividade por Departamento ---")
display(df_departamento)

folder_id = os.getenv("DRIVE_FOLDER_ID")
upload_df_to_drive(df_departamento, "Rotatividade por Departamento", folder_id, creds_path)
load_data_from_drive(folder_id)

dataframes_to_save = {"rotatividade_por_departamento" : df_departamento}
save_data_to_postgres(dataframes_to_save, db_url)



--- Tabela 1.1: Rotatividade por Departamento ---


,department,total_funcionarios,total_saidas,taxa_rotatividade_%
2,IT,174,37,21.264368
1,HR,211,44,20.853081
3,Marketing,198,36,18.181818
0,Finance,218,38,17.431193
4,Sales,199,34,17.085427


Processando tabela: rotatividade_por_departamento
Aviso: Tabela 'rotatividade_por_departamento' será criada, pois a limpeza falhou. (psycopg2.errors.UndefinedTable) relation "rotatividade_por_departamento" does not exist

[SQL: TRUNCATE TABLE rotatividade_por_departamento RESTART IDENTITY;]
(Background on this error at: https://sqlalche.me/e/20/f405)
Dados inseridos/atualizados com sucesso.


In [84]:
df_overtime = calcular_taxa_rotatividade(df, 'overtime')
print("\n--- Tabela 1.2: Rotatividade por Horas Extras ---")
display(df_overtime)

folder_id = os.getenv("DRIVE_FOLDER_ID")
upload_df_to_drive(df_departamento, "Rotatividade por Horas Extras", folder_id, creds_path)
load_data_from_drive(folder_id)

dataframes_to_save = {"rotatividade_por_horas_extras" : df_overtime}
save_data_to_postgres(dataframes_to_save, db_url)



--- Tabela 1.2: Rotatividade por Horas Extras ---


,overtime,total_funcionarios,total_saidas,taxa_rotatividade_%
0,False,512,100,19.531250
1,True,488,89,18.237705


Processando tabela: rotatividade_por_horas_extras
Aviso: Tabela 'rotatividade_por_horas_extras' será criada, pois a limpeza falhou. (psycopg2.errors.UndefinedTable) relation "rotatividade_por_horas_extras" does not exist

[SQL: TRUNCATE TABLE rotatividade_por_horas_extras RESTART IDENTITY;]
(Background on this error at: https://sqlalche.me/e/20/f405)
Dados inseridos/atualizados com sucesso.


In [ ]:
bins_tempo = [0, 1, 3, 7, 15, df['years_at_company'].max() + 1]
labels_tempo = ['< 1 ano (Risco Máximo)', '1-2 anos', '3-6 anos', '7-14 anos', '15+ anos']
df['tenure_group'] = pd.cut(df['years_at_company'], bins=bins_tempo, labels=labels_tempo, right=False)

df_tempo_casa = calcular_taxa_rotatividade(df, 'tenure_group')
print("\n--- Tabela 2.1: Rotatividade por Faixa de Tempo de Casa ---")
display(df_tempo_casa)

index_col_name = df_tempo_casa.index.name if df_tempo_casa.index.name is not None else df_tempo_casa.columns[0]

if index_col_name in df_tempo_casa.columns and df_tempo_casa[index_col_name].dtype.name == 'category':
    df_tempo_casa[index_col_name] = df_tempo_casa[index_col_name].astype('object')
elif df_tempo_casa.index.dtype.name == 'category':
    df_tempo_casa.index = df_tempo_casa.index.astype('object')
    df_tempo_casa = df_tempo_casa.reset_index()

df_tempo_casa_limpo = df_tempo_casa.fillna(0)

folder_id = os.getenv("DRIVE_FOLDER_ID")
upload_df_to_drive(df_tempo_casa_limpo, "Rotatividade por Faixa de Tempo de Casa", folder_id, creds_path)
load_data_from_drive(folder_id)

dataframes_to_save = {"rotatividade_por_tempo_de_casa" : df_tempo_casa_limpo}
save_data_to_postgres(dataframes_to_save, db_url)



--- Tabela 2.1: Rotatividade por Faixa de Tempo de Casa ---


/tmp/ipykernel_67223/4221894783.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agrupado = df.groupby(coluna_grupo)


,tenure_group,total_funcionarios,total_saidas,taxa_rotatividade_%
2,3-6 anos,135,31,22.962963
4,15+ anos,513,101,19.688109
3,7-14 anos,280,47,16.785714
1,1-2 anos,72,10,13.888889
0,< 1 ano (Risco Máximo),0,0,NaN


TypeError: Cannot setitem on a Categorical with a new category (0), set the categories first

In [34]:
df_salario_nivel = calcular_taxa_rotatividade(df, 'job_level')
print("\n--- Tabela 2.2: Rotatividade por Nível de Cargo (Job Level) ---")
display(df_salario_nivel)


--- Tabela 2.2: Rotatividade por Nível de Cargo (Job Level) ---


,job_level,total_funcionarios,total_saidas,taxa_rotatividade_%
3,4,205,43,20.975610
1,2,200,41,20.500000
2,3,206,40,19.417476
4,5,207,36,17.391304
0,1,182,29,15.934066


In [35]:
cols_analise = [
    'monthly_income', 
    'years_at_company',
    'job_satisfaction', 
    'work_life_balance',
    'relationship_with_manager', 
    'job_involvement',
    'distance_from_home'
]

# Agrupa pelo status de Attrition e calcula a média para cada fator
df_satisfacao_media = df.groupby('attrition')[cols_analise].mean().T 
df_satisfacao_media.columns = ['media_permanece_(false)', 'media_sai_(true)']
df_satisfacao_media.index.name = 'fator_de_risco'
df_satisfacao_media = df_satisfacao_media.reset_index()

print("\n--- Tabela 3: Média dos Fatores de Risco por Status de Attrition (True/False) ---")
display(df_satisfacao_media)


--- Tabela 3: Média dos Fatores de Risco por Status de Attrition (True/False) ---


,fator_de_risco,media_permanece_(false),media_sai_(true)
0,monthly_income,11471.595561,11621.349206
1,years_at_company,14.829840,15.317460
2,job_satisfaction,3.141800,3.190476
3,work_life_balance,2.477189,2.571429
4,relationship_with_manager,2.510481,2.555556
5,job_involvement,2.522811,2.417989
6,distance_from_home,24.695438,23.698413
